<a href="https://colab.research.google.com/github/dhyougit/NLP/blob/main/Question_ResponseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras

In [14]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

# Method1: Sequential
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# functional API
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 32)                1056      
                                                                 
 dense_15 (Dense)            (None, 10)                330       
                                                                 
Total params: 3466 (13.54 KB)
Trainable params: 3466 (13.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size = 128)

score = model.evalutate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 1s 3ms/step - loss: 12.1300
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.4422
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 13.1502
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 14.2637
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 15.8115
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 17.7379
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 20.1741
Epoch 8/10
8/8 [==============================] - 0s 3ms/step - loss: 23.0355
Epoch 9/10
8/8 [==============================] - 0s 8ms/step - loss: 26.5233
Epoch 10/10
8/8 [==============================] - 0s 2ms/step - loss: 30.4430


AttributeError: ignored

In [4]:
# Functional API is particularly useful to build multi-type input model

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedd_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedd_text)

question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question')

embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size,
                      activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer = 'rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# Now, let's inject data to the above model
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answers = to_categorical(answers)

model.fit([text, question], answers, epochs=10, batch_size=128)

model.fit({'text':text, 'question':question}, answers,
          epochs=10, batch_size=128)


Epoch 1/10
8/8 [==============================] - 4s 117ms/step - loss: 6.2148 - accuracy: 0.0020
Epoch 2/10
8/8 [==============================] - 1s 114ms/step - loss: 6.2098 - accuracy: 0.0070
Epoch 3/10
8/8 [==============================] - 1s 113ms/step - loss: 6.2061 - accuracy: 0.0050
Epoch 4/10
8/8 [==============================] - 1s 122ms/step - loss: 6.2021 - accuracy: 0.0080
Epoch 5/10
8/8 [==============================] - 1s 182ms/step - loss: 6.1971 - accuracy: 0.0050
Epoch 6/10
8/8 [==============================] - 2s 258ms/step - loss: 6.1888 - accuracy: 0.0060
Epoch 7/10
8/8 [==============================] - 1s 143ms/step - loss: 6.1689 - accuracy: 0.0060
Epoch 8/10
8/8 [==============================] - 1s 115ms/step - loss: 6.1341 - accuracy: 0.0060
Epoch 9/10
8/8 [==============================] - 1s 114ms/step - loss: 6.0984 - accuracy: 0.0060
Epoch 10/10
8/8 [==============================] - 1s 114ms/step - loss: 6.0690 - accuracy: 0.0060
Epoch 1/10
8/8 [===

In [18]:
text

array([[6126, 3139, 3191, ...,  851, 2583,  783],
       [4733, 9093, 3137, ..., 8640, 4410, 8352],
       [ 831, 2409, 5485, ..., 1831,  144,  341],
       ...,
       [3302,  938, 5491, ..., 5610, 2792, 8657],
       [4434, 1243, 2675, ..., 8655, 8139,  670],
       [5518, 8919, 5070, ..., 4999, 4475, 1441]])

In [19]:
text_vocabulary_size

10000

In [9]:
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])